# 크롤링 코드

In [ ]:
import csv
from csv import writer

# csv파일에다가 선수들의 정보, 전적을 append 시켜주는 함수 
# dict의 key와 csv파일의 컬럼을 자동으로 매칭시킴
def appendNewData_info(csv_file, dict_data):
    try:
        if dict_data['nationality'] == 'Japan' or dict_data['nationality'] == 'China':
            with open(csv_file, 'a', newline='', encoding = 'cp949') as f1:
                append_writer = csv.DictWriter(f1, fieldnames = ['name', 'division', 'rating', 'bouts', 'rounds', 'KOs', 'career', 'debut', 'vada', 'titles', 'ID#', 'birth name', 'sex', 'alias', 'age', 'nationality', 'stance', 'height', 'reach', 'residence', 'birth place', 'manager/agent', 'promoter', 'MMA'])
                append_writer.writerow(dict_data)
        else:
            with open(csv_file, 'a', newline='', encoding = 'utf-8') as f1:
                append_writer = csv.DictWriter(f1, fieldnames = ['name', 'division', 'rating', 'bouts', 'rounds', 'KOs', 'career', 'debut', 'vada', 'titles', 'ID#', 'birth name', 'sex', 'alias', 'age', 'nationality', 'stance', 'height', 'reach', 'residence', 'birth place', 'manager/agent', 'promoter', 'MMA'])
                append_writer.writerow(dict_data)
    except Exception as e:
        with open(csv_file, 'a', newline='', encoding = 'utf-8') as f1:
            append_writer = csv.DictWriter(f1, fieldnames = ['name', 'division', 'rating', 'bouts', 'rounds', 'KOs', 'career', 'debut', 'vada', 'titles', 'ID#', 'birth name', 'sex', 'alias', 'age', 'nationality', 'stance', 'height', 'reach', 'residence', 'birth place', 'manager/agent', 'promoter', 'MMA'])
            append_writer.writerow(dict_data)
        
        
def appendNewData_match_history(csv_file2, dict_data2):
    with open(csv_file2, 'a', newline='', encoding = 'utf-8') as f2:
        append_writer2 = csv.DictWriter(f2, fieldnames = ['date', 'player_1', 'player_2', 'match_result', 'match_rounds'])
        append_writer2.writerow(dict_data2)


# 필요한 모듈을 임포트 합니다
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
import warnings # 경고를 무시합니다.
warnings.filterwarnings('ignore')

# 변수 할당
page_num = input('몇 페이지에서 시작할까요? : ')
index_num = input('몇번째 선수부터 시작할까요? : ')
end_page = input('몇 페이지까지 크롤링할까요? : ')
timesFifty = (int(page_num)-1)*50
captcha = str('https://boxrec.com/recaptcha')
pattern = re.compile('\d{7}|\d{6}') # 정규식을 활용해서 id값이 숫자로만 이뤄진 6~7자리 인 값들만 가져옴
trial_num = 0 # capcha 떴을때 몇번 시도할지 정해줌
# 첫번째 계정
ID_1 = 'bigdataboxing@ruu.kr' 
PW_1 = 'Bigdata12'

#결과를 저장할 데이터프레임
df_player_info = pd.DataFrame() 
df_match_history = pd.DataFrame()

while timesFifty <= (int(end_page)-1)*50:
    
    try:
        #현재 설치되어있는 Chrome 버전과 동일한 Chromedriver을 다운받습니다
        driver = webdriver.Chrome('C:\\Users\\playdata\\chromedriver\\chromedriver.exe')
        driver.implicitly_wait(5) #로딩이 다 다음 페이지로 넘어갑니다
        driver.get("https://boxrec.com/en/login?error=limit")

        #로그인 합니다
        driver.find_element(By.XPATH,'//*[@id="username"]').send_keys(ID_1)
        driver.find_element(By.XPATH,'//*[@id="password"]').send_keys(PW_1)
        driver.find_element(By.XPATH,'//*[@id="pageOuter"]/div/div[2]/form/div[4]/button').click()

        #월드 랭킹으로 이동합니다
        pageOne = str('https://boxrec.com/en/ratings?sex=M&offset=' + str(timesFifty))
        driver.get(pageOne)
        
        while timesFifty <= (int(end_page)-1)*50:
            
            # 크롤링 부분

            for count in range(int(index_num), 50):
                dict_player_info = {}  # 선수 데이터가 들어가는 공 dict

                boxerN = int(count)
                boxerRanking = str('// *[ @ id = "se' + str(boxerN) + '"] / td[2] / a')
                driver.find_element(By.XPATH,boxerRanking).click()

                # 선수 정보 크롤링

                name = driver.find_element(By.XPATH, '//*[@id="pageOuter"]/div/div[4]/div[2]/table/tbody/tr[1]/td/h1')
                dict_player_info['name'] = name.text

                for i in range(3, 5):
                    for j in range(1, 12):
                        try:
                            Label = driver.find_element(By.XPATH,
                                                        f'//*[@id="pageOuter"]/div/div[4]/div[2]/table/tbody/tr[2]/td[2]/div[{i}]/table/tbody/tr[{j}]/td[1]/b')
                            value = driver.find_element(By.XPATH,
                                                        f'//*[@id="pageOuter"]/div/div[4]/div[2]/table/tbody/tr[2]/td[2]/div[{i}]/table/tbody/tr[{j}]/td[2]')
                            if ',' in value.text:
                                value_text = value.text.replace(',', ' & ')
                            else:
                                value_text = value.text

                            dict_player_info[Label.text] = value_text

                            rating = driver.find_element(By.XPATH,
                                                         '//*[@id="pageOuter"]/div/div[4]/div[2]/table/tbody/tr[2]/td[2]/div[3]/table/tbody/tr[3]/td[2]/a')
                            dict_player_info['rating'] = rating.text

                        except Exception as e:
                            continue

                # 데이터 전처리

                # rating : rating에서 #을 제거
                if 'rating' in dict_player_info.keys():
                    dict_player_info['rating'] = dict_player_info['rating'].lstrip('#')
                else:
                    pass

                # career는 경력으로 년수로 변환
                if 'career' in dict_player_info.keys():
                    if dict_player_info['career'].split('-')[0] != '? ':
                        dict_player_info['career'] = int(dict_player_info['career'].split('-')[1]) - int(
                            dict_player_info['career'].split('-')[0])
                    else:
                        pass
                else:
                    pass

                # titles : 수상경력이 있는 선수들은 개수로 변환
                if 'titles' in dict_player_info.keys():
                    dict_player_info['titles'] = len(dict_player_info['titles'].split('\n'))
                else:
                    pass
                # height, reach 는 cm인 값들만 뽑아옴
                if 'height' in dict_player_info.keys():
                    dict_player_info['height'] = dict_player_info['height'].split('/')[1].strip()
                else:
                    pass
                if 'reach' in dict_player_info.keys():
                    dict_player_info['reach'] = dict_player_info['reach'].split('/')[1].strip()
                else:
                    pass

                df_player_info = df_player_info.append(dict_player_info, ignore_index=True) # 데이터프레임에 dict_player_info 추가
                # dict_player_info의 값을 player_info.csv파일에 추가함

                appendNewData_info('player_info_hero.csv', dict_player_info)


                # 전적 데이터 크롤링

                html = driver.page_source # selenium에서 html페이지를 가져오는 메소드
                soup = BeautifulSoup(html, 'lxml')
                history = soup.select(".dataTable > tbody") # .dataTable => dataTable 클래스에서 
                                                            # >tbody => 태그가 tbody인 값들만 가져옴

                for i in range(len(history)):
                    id_num = pattern.findall(history[i].attrs['id']) # attrs 메소드를 사용해 bs4.element_result 타입인 history에서 id 값들을 가져오고 
                                                                    # 그 값들중 주어진 정규식 표현과 일치한 값들을 뽑아옴

                    # 날짜별로 구분된 id값들을 내가 원하는 정보의 XPATH에 format 형태로 대입하여 저장
                    # 아직 치워지지 않은 경기는 player_2가 없기때문에 오류 발생 일단 try,except구문으로 처리한 후 추후에 다시 고려
                    try:
                        date = driver.find_element(By.XPATH, '//*[@id={}]/td[2]/a'.format(*id_num))
                        player_1 = driver.find_element(By.XPATH, '//*[@id="pageOuter"]/div/div[4]/div[2]/table/tbody/tr[1]/td/h1')
                        player_2 = driver.find_element(By.XPATH, '//*[@id={}]/td[6]/a'.format(*id_num))
                        match_result = driver.find_element(By.XPATH, '//*[@id={}]/td[10]/div'.format(*id_num))
                        rounds = driver.find_element(By.XPATH, '//*[@id={}]/td[11]'.format(*id_num))

                        # 데이터프레임을 더 간편하게 만들기 위해서 dict형식으로 저장
                        dict_match_history = {'date' : date.text, 'player_1' : player_1.text, 'player_2' : player_2.text, 'match_result' : match_result.text, 'match_rounds' : rounds.text}

                        # 데이터프레임에 저장
                        df_match_history = df_match_history.append(dict_match_history, ignore_index=True)

                        # dict_match_history의 값을 match_history.csv파일에 추가함
                        appendNewData_match_history('match_history_hero.csv', dict_match_history)

                    except:
                        continue
                
                # 한 선수를 다 긁었으면 뒤로가기
                driver.back()

            # 다음 페이지로 이동합니다.
            timesFifty += 50
            world_ranking_page = str('https://boxrec.com/en/ratings?sex=M&offset=' + str(timesFifty))
            driver.get(world_ranking_page)
            index_num = 0 # 새로운 페이지가 시작될때 index를 0으로 초기화
        
        else:
            print('드디어 완료')
            dirver.quit()


    except Exception as e:
        if str(driver.current_url) == captcha:
            if trial_num <= 3:
                print('capcha 발생')
                timesFifty = timesFifty
                index_num = count
                # 두번째 계정
                ID_1 = 'Boxing@ruu.kr'
                PW_1 = 'Boxing12'
                trial_num += 1
                time.sleep(10) # 혹시 모를 capcha 풀 시간
                driver.quit()
                continue

            elif trial_num <= 6:
                print('capcah 벗어나기 실패')
                print('종료된 페이지:' + str((timesFifty // 50) + 1))
                print('다시시작해야 할 index : {}'.format(count))
                # 세번째 계정
                ID_1 = 'tjrdnw12@gmail.com'
                PW_1 = 'Bigdata12'
                trial_num += 1
                timesFifty = timesFifty
                index_num = count
                time.sleep(10)
                driver.quit()
            
            elif trial_num <= 10:
                print('capcah 벗어나기 실패')
                print('종료된 페이지:' + str((timesFifty // 50) + 1))
                print('다시시작해야 할 index : {}'.format(count))
                # 세번째 계정
                ID_1 = 'bigdataboxing@ruu.kr' 
                PW_1 = 'Bigdata12'
                trial_num += 1
                timesFifty = timesFifty
                index_num = count
                time.sleep(10)
                driver.quit()
            
            else:
                break
            
        else:
            print("비정상적인 종료")
            print('오류종류 : {}'.format(e), '\n')
            print('종료된 페이지:' + str((timesFifty // 50) + 1))
            print('다시시작해야 할 index : {}'.format(count))
            timesFifty = timesFifty
            index_num = count
            driver.quit()

            
else:
    print('드디어 완료')
    dirver.quit()


# 크롤링 된 파일들 Concat 및 데이터 전처리

In [2]:
# 데이터프레임 concat 함수
def dfConcat(*args):
    df_concat = pd.concat(list(args), axis = 0)
    df_concat.reset_index(drop = True, inplace = True)
    return df_concat

# 중복되는 데이터 검사
def duplicated(dataframe, *args):
    dup = dataframe.duplicated(list(args))
    dup = pd.DataFrame(dup)
    print("중복된 데이터 개수 : ",len(dup.loc[dup[0]]))
    
# 중복되는 데이터 삭제
def duplicatedDrop(dataframe, *args):
    dataframe = dataframe.drop_duplicates(list(args), keep = 'first')
    dataframe.reset_index(drop = True, inplace = True)
    
    return dataframe

## 데이터 프레임 병합
# 컬럼명 재정의
def renameCol(dataframe):
    for i in dataframe.columns:
        if '_x' in i:
            dataframe.rename(columns = {i : i.replace('_x', '_player_1')}, inplace = True)
        if '_y' in i:
            dataframe.rename(columns = {i : i.replace('_y', '_player_2')}, inplace = True)
    return dataframe

# 데이터 병합(복싱데이터 기준)
def mergeDf(df_match_history, df_player_info):
    df_player_1 = pd.merge(df_match_history, df_player_info, left_on='player_1', right_on='name')
    df_player_2 = pd.merge(df_match_history, df_player_info, left_on = 'player_2', right_on='name')
    final_df = pd.merge(df_player_1,df_player_2, on = ['date', 'player_1', 'player_2', 'match_result', 'match_rounds'])
    renameCol(final_df)
    return final_df

In [ ]:
# 데이터프레임 얼마나 볼지
# pd.options.display.max_rows = 60
# pd.options.display.max_columns = 20
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# 서영웅
df_match_history_hero = pd.read_csv('match_history_hero.csv')

df_player_info_hero = pd.read_csv('player_info_hero.csv', encoding = 'cp949')

# 이지수
df_player_info_soo = pd.read_csv('player_info_soo.csv', encoding = 'cp949')

df_match_history_soo = pd.read_csv('match_history_soo.csv', encoding = 'cp949')

# 이신엽
df_boxer_data = pd.read_csv('boxer_data.csv')

df_player_info_shin = pd.read_csv('player_info_shin.csv', encoding='latin1')

df_match_history_shin = pd.read_csv('match_history_shin.csv', encoding='latin1')

# 이정범

# 이나현
df_match_history_na = pd.read_csv('match_history_nn.csv', encoding = 'cp949')

df_player_info_na = pd.read_csv('player_info_nn.csv', encoding = 'cp949')

In [ ]:
# 파일들 Concat
df_match_history = dfConcat(df_match_history_hero, df_match_history_soo, df_match_history_na, df_match_history_shin)
df_player_info = dfConcat(df_player_info_hero, df_player_info_soo, df_player_info_na, df_player_info_shin)

In [ ]:
# 중복된 데이터 검사
duplicated(df_player_info, 'name')
duplicated(df_match_history, 'date', 'player_1', 'player_2', 'match_result', 'match_rounds')

In [ ]:
df_player_info = duplicatedDrop(df_player_info, 'name')
df_match_history = duplicatedDrop(df_match_history, 'date', 'player_1', 'player_2', 'match_result', 'match_rounds')

In [ ]:
duplicated(df_player_info, 'name')
duplicated(df_match_history, 'date', 'player_1', 'player_2', 'match_result', 'match_rounds')

In [3]:
# 데이터 전처리 (컬럼별)

# 필요없는 컬럼 drop
# 'ID#', 'vada', 'birth name', 'sex', 'alias', 'birth place', 'manager/agent', 'promoter', 'MMA'
# residence : 나라만 저장
# ID는 정수로 하면 머신러닝때 영향을 끼칠 수 있기 때문에 str타입으로 변경(머신러닝때는 drop 예정)
# rating : int로 변환하고 총인원 중 몇 %에 위치하는지 계산 & 아래에 몇명이 있는지로 변환 (e.g 수능 성적 받으면 1등급은 백분위 수가 96으로 되어있는것과 동일한 원리)
# reach와 height는 단위를 제거하고 int값으로 변환
# KOs : 단위값 제거하고 float으로 변환
# career : career가 0이면 일단 debut의 연도와 현재 연도를 빼서 career를 구함 => debut 컬럼은 필요없으니 drop
# titles, age : int로 형변환

# division
# 미니멈, 라이트플라이, 플라이, 슈퍼플라이 : 1
# 밴텀, 슈퍼밴텀, 페더, 슈퍼페터 : 2
# 라이트, 슈퍼라이트, 웰터, 슈퍼웰터 : 3
# 미들, 슈퍼미들, 라이트헤비 : 4
# 크루저, 헤비 : 5

# nationality, residence 둘중 하나라도 밑의 리스트에 존재하면
# li1 = ['USA', 'Mexico', 'United Kingdom'] => 3
# li2 = ['Russia', 'Japan', 'Ukraine', 'Philippines'] => 2
# li3 = ['Cuba', 'South Africa', 'Argentina'] => 1

# 해당 plus_point로 변환
# stance는 OneHotEncoding으로 orthodox / southpaw / unknown_position 으로 존재하면 1, 존재하지 않으면 0으로 인코딩 후 stance 컬럼 drop
# 머신러닝을 위해서 object 형태인 컬럼들을 int or float 으로 형변환 해줌

In [ ]:
# 필요없는 컬럼 drop & 복사
df_player_info2 = df_player_info.drop(['ID#', 'vada', 'birth name', 'sex', 'alias', 'birth place', 'manager/agent', 'promoter', 'MMA'], axis = 1)

In [ ]:
# fillna로 null값을 채워줌
df_player_info2.fillna(0, inplace = True)

In [ ]:
# 데이터 전처리 (컬럼별)

# titles, age, residence, career, debut 컬럼

from datetime import datetime
current_time = datetime.now()

df_player_info2['titles'] = df_player_info2['titles'].astype('int')
df_player_info2['age'] = df_player_info2['age'].astype('int')

for i in range(len(df_player_info2.index)):
    
    # residence
    if df_player_info2['residence'][i] != 0:
        df_player_info2['residence'][i] = df_player_info2['residence'][i].split('&')[-1].strip(' ')
    else:
        pass

    #career
    if df_player_info2['career'][i] == 0 and df_player_info2['debut'][i] != 0:
        df_player_info2['career'][i] = current_time.year - int(df_player_info2['debut'][i].split('-')[0])
    elif df_player_info2['career'][i] == '? -2022' or df_player_info2['career'][i] == '? -2021':
        df_player_info2['career'][i] = current_time.year - int(df_player_info2['debut'][i].split('-')[0])
    else:
        pass
        
df_player_info2.drop(['debut'], axis = 1, inplace = True)

In [ ]:
# df_player_info_gui = df_player_info2.copy()
# df_player_info_gui.to_csv('player_info_gui.csv', encoding = 'utf-8', index = False)

## 여기까지 전처리한게 player_info_gui Table

In [ ]:
for i in range(len(df_player_info2.index)):
    
    # rating
    if df_player_info2['rating'][i] != 0:
        df_player_info2['rating'][i] = df_player_info2['rating'][i].replace(',', '')
        df_player_info2['rating'][i] = round(1 - (int(df_player_info2['rating'][i].split('/')[0]) / int(df_player_info2['rating'][i].split('/')[1])), 5)*100
    else:
        pass
    
    # KOs
    if df_player_info2['KOs'][i] != 0:
        df_player_info2['KOs'][i] = float(df_player_info2['KOs'][i].rstrip('%'))
    else:
        pass
    
    # reach, height
    if df_player_info2['height'][i] != 0:
        df_player_info2['height'][i] = int(df_player_info2['height'][i].rstrip('cm'))
    else:
        pass
    
    if df_player_info2['reach'][i] != 0:
        df_player_info2['reach'][i] = int(df_player_info2['reach'][i].rstrip('cm'))
    else:
        pass

In [ ]:
# division
for i in range(len(df_player_info2.index)):
    if df_player_info2['division'][i] != 0:
        if df_player_info2['division'][i] == 'minimum' or df_player_info2['division'][i] == 'light fly' or df_player_info2['division'][i] == 'fly' or df_player_info2['division'][i] == 'super fly':
            df_player_info2['division'][i] = 1
        elif df_player_info2['division'][i] == 'bantam' or df_player_info2['division'][i] == 'super bantam' or df_player_info2['division'][i] == 'feather' or df_player_info2['division'][i] == 'super feather':
            df_player_info2['division'][i] = 2
        elif df_player_info2['division'][i] == 'light' or df_player_info2['division'][i] == 'super light' or df_player_info2['division'][i] == 'welter' or df_player_info2['division'][i] == 'super welter':
            df_player_info2['division'][i] = 3
        elif df_player_info2['division'][i] == 'middle' or df_player_info2['division'][i] == 'super middle' or df_player_info2['division'][i] == 'light heavy':
            df_player_info2['division'][i] = 4
        elif df_player_info2['division'][i] == 'cruiser' or df_player_info2['division'][i] == 'heavy':
            df_player_info2['division'][i] = 5
    else:
        pass

In [ ]:
# plus points(파생변수)

li1 = ['USA', 'Mexico', 'United Kingdom']
li2 = ['Russia', 'Japan', 'Ukraine', 'Philippines']
li3 = ['Cuba', 'South Africa', 'Argentina']
df_player_info2['plus_points'] = 0

for i in range(len(df_player_info2.index)):
    if df_player_info2['residence'][i] in li1 or df_player_info2['nationality'][i] in li1:
        df_player_info2['plus_points'][i] = 3
    elif df_player_info2['residence'][i] in li2 or df_player_info2['nationality'][i] in li2:
        df_player_info2['plus_points'][i] = 2
    elif df_player_info2['residence'][i] in li3 or df_player_info2['nationality'][i] in li3:
        df_player_info2['plus_points'][i] = 1
    else:
        df_player_info2['plus_points'][i] = 0
df_player_info2.drop(['nationality', 'residence'],axis = 1, inplace = True)

In [ ]:
# ONeHotEncoding (stance)
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
label = df_player_info2['stance'].astype('str')
ohe.fit(label.values.reshape(-1,1))
one_hot_encoded = ohe.transform(label.values.reshape(-1,1))
ohe_df = pd.DataFrame(one_hot_encoded, columns = ohe.categories_[0])

ohe_df

In [ ]:
df_player_info2 = pd.concat([df_player_info2, ohe_df], axis = 1)
df_player_info2.rename(columns = {'0' : 'unknown_position'}, inplace = True)
df_player_info2.drop(['stance'], axis = 1, inplace = True)

In [ ]:
# 머신러닝을 위한 컬럼 형변환
df_player_info2 = df_player_info2.astype({'division' : 'int', 'career' : 'int', 'height' : 'int', 'reach' : 'int'})
df_player_info2 = df_player_info2.astype({'KOs' : 'float', 'rating' : 'float'})

In [ ]:
# df_player_info_preprocessing = df_player_info2.copy()
# df_player_info_preprocessing.to_csv('player_info_preprocess.csv', encoding = 'utf-8', index = False)

## 여기까지 전처리 한게 player_info_preprocess Table

# 데이터프레임 merge (match_history, player_info)

In [ ]:
boxing_df = mergeDf(df_match_history, df_player_info2).drop(['name_player_1', 'name_player_2'], axis = 1) # 21097 rows

# 머신러닝을 위한 데이터전처리

In [ ]:
boxing_df['player_1_result'] = 0
boxing_df['player_2_result'] = 0

for i in range(len(boxing_df.index)):
    if boxing_df['match_result'][i].split('-')[0] == 'W':
        boxing_df['player_1_result'][i] = 'W'
        boxing_df['player_2_result'][i] = 'L'
    elif boxing_df['match_result'][i].split('-')[0] == 'L':
        boxing_df['player_1_result'][i] = 'L'
        boxing_df['player_2_result'][i] = 'W'
    elif boxing_df['match_result'][i].split('-')[0] == 'D':
        boxing_df['player_1_result'][i] = 'D'
        boxing_df['player_2_result'][i] = 'D'
    else:
        boxing_df['player_1_result'][i] = 'unknown'
        boxing_df['player_2_result'][i] = 'unknown'   

In [ ]:
# boxing_df.to_csv('boxing_merge.csv', encoding = 'utf-8', index = False)

## 여기까지 전처리한게 boxing_merge Table

# 머신러닝

In [1]:
import pandas as pd
import pymysql.cursors
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import sklearn.metrics as mt

result = pd.read_csv('boxing_merge.csv')

def df_ML():

    df_features = result.drop(['date', 'player_1', 'player_2', 'match_result', 'match_rounds', 'player_1_result', 'player_2_result'], axis = 1)
    df_label = result['player_1_result']
    
    # Label 값 LabelEncoding
    
    encoder = LabelEncoder()
    encoder = encoder.fit(df_label)
    df_label_encoded = encoder.transform(df_label)
    
    # Features 값 MinMaxScale
    
    scaler = MinMaxScaler()
    scaler.fit(df_features)
    df_features_scaled = scaler.transform(df_features)
    
    # Split
    X_train, X_test, y_train, y_test = train_test_split(df_features_scaled, df_label_encoded, test_size=0.3)
    
    # Optuna에서 뽑은 하이퍼파라미터
    param = {
        'max_depth': 13,
        'subsample': 1.0,
        'n_estimators': 4800,
        'eta': 0.06999999999999999,
        'reg_alpha': 27,
        'reg_lambda': 87,
        'min_child_weight': 17,
        'colsample_bytree': 0.9110221681181114
    }

    xgb = XGBClassifier(**param)
    xgb.fit(X_train, y_train, early_stopping_rounds = 400, eval_set = [(X_test, y_test)], eval_metric = 'auc', verbose = False)
    pred = xgb.predict(X_test)
    print('예측 정확도 : {0:.4f}'.format(mt.accuracy_score(y_test, pred)))




df_ML()

C:\Users\playdata\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\playdata\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


예측 정확도 : 0.8125
